<a href="https://colab.research.google.com/github/erictong0/FrederickLab2023/blob/main/Linear%20Regression/Custom%20Optimization/230630_Combining_Alpha_%2B_Beta_Pixelated_Linear_regression_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# irrelevant

In [ ]:
#imports
!pip install bio --upgrade
!pip install scipy

In [ ]:
#imports for timing things
!pip install line_profiler
%load_ext line_profiler

In [ ]:
import sys
import glob
import math
import argparse
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as pltticker
import matplotlib.cm as cm
from matplotlib.patches import Rectangle
import csv
from matplotlib import colors
import re
import pandas as pd
from numpy import savetxt
from sklearn.linear_model import LinearRegression
from statistics import mean
import statsmodels.api as sm
from Bio.Data.IUPACData import protein_letters_1to3 as one2three
import scipy.optimize as spo
from pandas.core.internals.construction import ma
from sklearn.linear_model import Lasso

In [ ]:
#works to load information in variables before the program

fwhm = 1.5  # Full width at half maximum
# Calculate the standard deviation from the FWHM
c = fwhm / (2 * np.sqrt(2 * np.log(2)))

def get_row_floats(data, row):
  return list(map(lambda x: float(x[row]), data))

xArray = []
for i in range(96):
  b = i / 4 + 54
  xArray.append(np.linspace(b - 3 * c, b + 3 * c, 100))

oData = []
data = []
with open("T_10K.out", mode="r") as infile: #open data file and read contents
  reader = csv.reader(infile, delimiter="\n")
  for row in reader:
    if len(row) == 1:
      data.append(re.split('\t', row[0])[2:])
    else:
      data.append(row)

tempData = []
for i in range(6):
  tempData.append(np.array(get_row_floats(data[1:], i)))

oData.append(tempData)

data = []
tempData = []
count = 0
with open("spotlight 2022.csv", mode="r") as infile: #open data file and read contents
  chunk_size = 1000
  while True:
      chunk = infile.readlines(chunk_size)
      if not chunk:
        break
      for line in chunk:
        reader = csv.reader(infile)
        for row in reader:
          count += 1
          if len(row) == 1:
            tempData.append(re.split(r',', row[0]))
          else:
            tempData.append(row)

data.append(get_row_floats(tempData,0)[1:])
data.append(get_row_floats(tempData,1)[1:])

oData.append(data)

DATAINPUT = oData

In [ ]:
xScale = 5
yScale = 5

def main(inData, aa, rangeChoices):
  aa1 = aa

  phi = inData[0][0]
  psi = inData[0][1]
  ca = inData[0][2]
  cb = inData[0][3]
  c = inData[0][4]
  n = inData[0][5]



  ca_exp = inData[1][0] #def x as Ca shift
  intensity = inData[1][1]


  atom_key={
            "ca":0,
            "cb":1,
            "co":2,
            "n":3,
          }

  nonCys_nonGly_aas={"A", "D", "E", "F", "H", "I", "K", "L", "M",
                    "N", "P", "Q", "R", "S", "T", "V", "Y", "W"}

  nonCys_aas={"A", "D", "E", "F", "G", "H", "I", "K", "L", "M",
              "N", "P", "Q", "R", "S", "T", "V", "Y", "W"}

  na=np.nan

  aa_spec_ranges={ #ranges of interest for chemical shifts
                  #aa1 Ca      Cb      C         N
                  # Ca & Cb adjusted to 17ppm to match Fig5
                 'A':[[11,28],[203,234],[170,184],[112,133]],
                  'R':[[47,64],[22,39],[169,183],[112,133]],
                  'N':[[44,61],[30,47],[168,182],[106,127]],
                  'D':[[45,62],[33,50],[169,183],[109,130]],
  #               'C':[[50,67],[22,39],[168,182],[105,126]],
                  'Q':[[47,64],[22,39],[169,183],[108,129]],
                  'E':[[49,66],[22,39],[169,183],[109,130]],
                  'G':[[36,53],[203,234],[167,181],[ 98,119]],
                  'H':[[49,66],[23,40],[168,182],[107,128]],
                  'I':[[54,71],[31,48],[168,182],[109,130]],
                  'L':[[45,62],[33,50],[170,184],[110,131]],
                  'K':[[49,66],[26,43],[169,183],[109,130]],
                  'M':[[47,64],[26,43],[169,183],[108,129]],
                  'F':[[50,67],[31,48],[168,182],[107,128]],
                  'P':[[57,74],[25,42],[170,184],[ na, na]],
                  'S':[[50,67],[105,139],[55,72],[167,181],[104,125]],
                  'T':[[54,70],[60,77],[168,182],[102,123]],
                  'W':[[48,65],[21,38],[169,183],[109,130]],
                  'Y':[[50,67],[30,47],[168,182],[108,129]],
                  'V':[[54,71],[25,42],[169,183],[108,129]],
                  }

  ####Changed this part####
  rchoice = rangeChoices

  ### extract plot subregions ###
  # "r" stands for ramachandran
  # regions: beta-strand, polyproline helix ii
  #          beta-turn I, alpha-helix,
  #          left handed helix (all phi > 0)
  # !!! top and left corner are always >= or <=
  #     unless they reach the bottom/right axis
  #     (i.e. betaB, ppiiB, left)

  #filter beta regions

  def makeMask(a, b, c, d):
    return np.logical_and(np.logical_and(phi >= a, phi <= b), np.logical_and(psi >= c, psi <= d))

  masks = {}
  datas = {}
  cs_ca = {}
  cs_cb = {}
  cs_co = {}

  for i in range(int(180/xScale)):
    for j in range(int(360/yScale)):
      name = f"{chr(i+65)}{(j+1):02d}"
      mask = makeMask(i*xScale-180, (i+1)*xScale-180, 180-(j+1)*yScale, 180-j*yScale)
      #print((i*10-180, i*10-170, 170-j*10, 180-j*10))
      cs_ca[name] = ca[mask]
      cs_cb[name] = cb[mask]
      cs_co[name] = c[mask]

  name = "rturni"
  mask = makeMask(0, 180, -180, 180)
  cs_ca[name] = ca[mask]
  cs_cb[name] = cb[mask]
  cs_co[name] = c[mask]
  #cs_ca = list(cs_ca) ???

  cs_all = {'ca':cs_ca,
            'cb':cs_cb,
            'c' :cs_co,
            }
  for key in cs_ca.keys():
    #print(cs_cb[key])
    cs_ca[key] = np.append(cs_ca[key], cs_cb[key])

  bin_width=0.25
  mtick_spacing=5
  #fig, axs = plt.subplots(1, 1, sharey=False, tight_layout=True,figsize=(10,10)) #35826579

  x_lims = np.array(aa_spec_ranges[aa1][0])
  y_lims = np.array(aa_spec_ranges[aa1][1])

  ###Changed this like as well
  x_edges = np.arange(x_lims[0],y_lims[1]+bin_width,bin_width) #create an array from one limit to the next using bin widths
  y_edges=np.arange(y_lims[0],y_lims[1]+bin_width,bin_width)
  hw=bin_width/2
  center_bins_x=np.arange(x_lims[0]+hw,x_lims[1]+hw,bin_width)
  center_bins_y=np.arange(y_lims[0]+hw,y_lims[1]+hw,bin_width)
  inputs = []
  h_lists = []
  h_list = []
  c = fwhm / (2 * np.sqrt(2 * np.log(2)))
  for key in cs_ca:
    H, bins = np.histogram(cs_ca[key],bins=(x_edges),density=False)
    for j in range(len(H)):
      a = H[j]  # Maximum value
      b = bins[j]  # Peak position

      x = xArray[round(b*4-216)]

      # Calculate the y values for the curve using the Gaussian function
      y = a * np.exp(-(x - b)**2 / (2 * c**2))
      histogram, bins = np.histogram(x, bins=x_edges, density=False, weights = y)
      h_list.append(histogram)
    h_lists = sum(h_list)
    inputs.append(h_lists)

  ###Changed
  H_exp, bins = np.histogram(ca_exp,bins=(x_edges),density=False, weights = intensity)
  target = (H_exp)

  tempRet = inputs
  tempArr = []
  for element in inputs:
    isSame = False
    for element2 in tempArr:
      if np.all(element == element2):
        isSame = True
    if isSame:
      pass
    else:
      tempArr.append(element)
  inputs = tempArr


  x = inputs
  y = target
  return (x, y, cs_ca, tempRet)


def func3P(dict):
  r, s, cs_ca, tempRet = main(DATAINPUT, "T", dict)
  #print(r.summary())
  return (r, s, cs_ca, tempRet)
  return (r.summary(), s, e)

x, y, cs_ca, rawInput = func3P({})
#resultString, cs_ca_list, H_exp = func3P({})



#Insert saved coefficients here


In [ ]:

copied = """1.26726875e+02 2.67544837e+06 1.20211626e+06 6.39223430e+05
 1.08352424e+05 2.15098434e+05 1.28238978e+05 5.07732835e+04
 1.09164948e+04 1.10374382e+02 2.90777923e+03 4.55497311e+03
 3.70311107e+03 4.40445121e+02 2.09025558e+02 5.21515154e+01
 6.94093843e+01 1.09948175e+02 5.39041439e+02 4.77514524e+03
 8.02440644e+02 1.23776772e+03 2.93398044e+03 2.37085296e+03
 8.10599806e+02 5.82707058e+02 3.42675627e+02 1.98932856e+02
 1.85569892e+02 4.72874672e+02 9.82356046e+02 5.58533975e+02
 4.00986187e+02 4.74697360e+02 5.94091209e+02 3.16124016e+02
 2.39516529e+02 1.62532728e+02 1.26345827e+02 8.36121915e+01
 5.98090931e+01 6.25760417e+01 7.57277736e+01 1.11905975e+02
 1.12068666e+02 1.36940678e+02 1.54903090e+02 1.63345399e+02
 1.61603367e+02 8.32393019e+01 4.39541034e+01 2.84956462e+01
 2.51365199e+01 2.47552881e+01 3.87249259e+01 3.76471555e+01
 4.28954398e+01 3.84434448e+01 3.38706162e+01 3.25423873e+01
 3.39808811e+01 3.07023980e+01 3.45964640e+01 3.61119836e+01
 3.95606638e+01 1.33414888e+02 7.12359536e+02 2.83481100e+02
 2.22099978e+01 2.04954311e+01 2.42059817e+01 3.10351883e+01
 3.67424510e+01 2.11810984e+01 1.85300724e+01 1.57146430e+01
 1.44545092e+01 1.37030655e+01 1.35764827e+01 1.34647301e+01
 1.34487307e+01 1.27596553e+01 1.25992129e+01 1.20139670e+01
 1.18168820e+01 1.17035008e+01 1.15829451e+01 1.15147831e+01
 1.16180675e+01 1.17440026e+01 1.27163640e+01 1.46205226e+01
 1.59365508e+01 1.62603211e+01 1.84726622e+01 2.08690987e+01
 2.42655183e+01 2.65893876e+01 1.37236619e+01 1.18174560e+01
 1.14436430e+01 1.11329888e+01 1.07913533e+01 1.07172645e+01
 1.06714083e+01 1.05461000e+01 1.06305668e+01 1.07434554e+01
 1.09322367e+01 1.09741663e+01 1.09876177e+01 1.10541310e+01
 1.10539611e+01 1.09854205e+01 1.08920116e+01 1.07821511e+01
 1.07303958e+01 1.06171088e+01 1.04580011e+01 1.00963685e+01
 9.90302995e+00 9.79006024e+00 9.33948846e+00 9.30373892e+00
 9.53115678e+00 9.95611193e+00 1.02554243e+01 1.05448117e+01
 1.10482761e+01 1.11810623e+01 1.11009134e+01 1.09857185e+01
 1.10521795e+01 1.10351031e+01 1.08574341e+01 1.07984058e+01
 1.02320551e+01 9.34758775e+00 9.11679458e+00 8.66454778e+00
 8.45298997e+00 8.29197945e+00 8.16997405e+00 8.38242645e+00
 8.88566403e+00 9.26234066e+00 9.40410086e+00 9.43195082e+00
 9.44242022e+00 9.46654303e+00 9.38189100e+00 9.35502001e+00
 9.27276043e+00 8.62017215e+00 8.01371723e+00 7.28062575e+00
 6.78266491e+00 6.57116271e+00 6.35992296e+00 6.19295471e+00
 6.15911031e+00 5.88728822e+00 5.63090188e+00 5.54414688e+00
 5.54093575e+00 5.54012237e+00 5.53773023e+00 5.53285760e+00
 5.53449384e+00 5.53606959e+00 5.53919045e+00 5.91854692e+00
 6.56142694e+00 6.65396442e+00 6.92016229e+00 6.89028303e+00
 6.91266894e+00 6.86807316e+00 6.52346018e+00 6.31072743e+00
 5.70547308e+00 5.47985343e+00 5.46482510e+00 5.45304400e+00
 5.44743125e+00 5.44348706e+00 5.44184991e+00 5.44133771e+00
 5.44372318e+00 5.44559112e+00 5.44607875e+00 5.44554196e+00
 5.44506223e+00 5.44209871e+00 5.43810725e+00 5.42637138e+00
 5.42056651e+00 5.41257988e+00 5.39518966e+00 5.38335766e+00
 5.37269987e+00 5.36319371e+00 5.35305916e+00 5.35185765e+00
 5.34972108e+00 5.34566764e+00 5.34252128e+00 5.33989856e+00
 5.33693704e+00 5.33493699e+00 5.33496983e+00 5.34130680e+00
 5.34377032e+00 5.34412758e+00 5.34415967e+00 5.34521117e+00
 5.34356616e+00 5.34215491e+00 5.34009351e+00 5.33377769e+00
 5.33345944e+00 5.31646678e+00 5.30054141e+00 5.29383177e+00
 5.28582616e+00 5.28103859e+00 5.27529799e+00 5.27469790e+00
 5.27475520e+00 5.27748504e+00 5.27768752e+00 5.27705269e+00
 5.27264482e+00 5.26987267e+00 5.24237735e+00 5.23365360e+00
 5.20703798e+00 5.18582357e+00 5.17523998e+00 5.16848749e+00
 5.15952889e+00 5.14307973e+00 5.14007318e+00 5.13630804e+00
 5.13418635e+00 5.13361029e+00 5.13060223e+00 5.12696051e+00
 5.12504964e+00 5.12408441e+00 5.12365278e+00 5.12318117e+00
 5.12713566e+00 5.13557262e+00 5.13741752e+00 5.13896919e+00
 5.14211518e+00 5.14425274e+00 5.14663443e+00 5.14550968e+00
 5.14413629e+00 5.14372704e+00 5.14238023e+00 5.11488944e+00
 5.10968506e+00 5.10237875e+00 5.08943567e+00 5.08317792e+00
 5.08050637e+00 5.08070431e+00 5.08165253e+00 5.08267443e+00
 5.08185427e+00 5.08024349e+00 5.07305173e+00 5.05853400e+00
 5.04090367e+00 5.01915042e+00 5.00278431e+00 4.98690252e+00
 4.96598502e+00 4.95625377e+00 4.94244730e+00 4.93453652e+00
 4.93356287e+00 4.93290503e+00 4.93233437e+00 4.93345225e+00
 4.93365817e+00 4.93323682e+00 4.93158997e+00 4.93158491e+00
 4.92894535e+00 4.92917463e+00 4.92495326e+00 4.92083362e+00
 4.91117181e+00 4.89650745e+00 4.87449575e+00 4.86959768e+00
 4.86649144e+00 4.86684334e+00 4.86761038e+00 4.87047103e+00
 4.87178921e+00 4.87064935e+00 4.86093981e+00 4.85257450e+00
 4.83588877e+00 4.82072628e+00 4.81186085e+00 4.79769233e+00
 4.76114762e+00 4.75914206e+00 4.75841805e+00 4.74881809e+00
 4.74861175e+00 4.74713356e+00 4.74683868e+00 4.74641816e+00
 4.75163364e+00 4.75345996e+00 4.75147457e+00 4.74678956e+00
 4.74298371e+00 4.73701423e+00 4.73491079e+00 4.73216906e+00
 4.72496200e+00 4.72284696e+00 4.71886726e+00 4.70875490e+00
 4.70839040e+00 4.70780706e+00 4.70891106e+00 4.70909983e+00
 4.71004025e+00 4.71096362e+00 4.71016477e+00 4.70780729e+00
 4.70287414e+00 4.69367297e+00 4.68042235e+00 4.65840641e+00
 4.63458364e+00 4.61928193e+00 4.60725429e+00 4.59199818e+00
 4.58489330e+00 4.58280506e+00 4.58158707e+00 4.58112703e+00
 4.58053955e+00 4.58054344e+00 4.58069383e+00 4.58198057e+00
 4.58233646e+00 4.57775272e+00 4.55382851e+00 4.54484526e+00
 4.53665216e+00 4.53453836e+00 4.53250116e+00 4.52741583e+00
 4.52487820e+00 4.52441613e+00 4.52517350e+00 4.52741433e+00
 4.53041163e+00 4.53105278e+00 4.53280257e+00 4.53271214e+00
 4.53227653e+00 4.53021940e+00 4.51386898e+00 4.49884840e+00
 4.48638291e+00 4.46905861e+00 4.45923473e+00 4.45149025e+00
 4.44730769e+00 4.44067258e+00 4.44036148e+00 4.44071140e+00
 4.44088380e+00 4.44119773e+00 4.44142387e+00 4.44285661e+00
 4.44422736e+00 4.44619724e+00 4.44354741e+00 4.42879632e+00
 4.41645739e+00 4.39922181e+00 4.38527955e+00 4.37891105e+00
 4.37726816e+00 4.37388794e+00 4.37362689e+00 4.37381149e+00
 4.37555309e+00 4.37621576e+00 4.37653513e+00 4.37628626e+00
 4.37626799e+00 4.37510952e+00 4.37380850e+00 4.36942672e+00
 4.35520619e+00 4.34542997e+00 4.33076894e+00 4.31931304e+00
 4.30514979e+00 4.30082956e+00 4.29443268e+00 4.29254959e+00
 4.29276120e+00 4.29301522e+00 4.29327896e+00 4.29373640e+00
 4.29421412e+00 4.29595696e+00 4.29644791e+00 4.28748192e+00
 4.26789166e+00 4.24024160e+00 4.19012480e+00 4.17938331e+00
 4.17283499e+00 4.16848793e+00 4.16703218e+00 4.16665378e+00
 4.16832877e+00 4.17235070e+00 4.17295342e+00 4.17386500e+00
 4.17407953e+00 4.17187593e+00 4.17192387e+00 4.17150430e+00
 4.17133269e+00 4.16864191e+00 4.16011228e+00 4.14799471e+00
 4.14112327e+00 4.12771063e+00 4.12048062e+00 4.11896349e+00
 4.11631697e+00 4.11452066e+00 4.11326114e+00 4.11356690e+00
 4.11331678e+00 4.11301509e+00 4.11295186e+00 4.11294988e+00
 4.11417131e+00 4.10271137e+00 4.08810762e+00 4.05835402e+00
 4.02217846e+00 4.00419519e+00 3.99790701e+00 3.99675773e+00
 3.99630641e+00 3.99631100e+00 3.99631422e+00 3.99748899e+00
 3.99897253e+00 3.99935301e+00 3.99995792e+00 3.99936272e+00
 3.99818567e+00 3.99604585e+00 3.99579156e+00 3.99467454e+00
 3.99423022e+00 3.99292011e+00 3.98943969e+00 3.98564203e+00
 3.98224315e+00 3.97489086e+00 3.96981505e+00 3.96724400e+00
 3.96625601e+00 3.96626076e+00 3.96592144e+00 3.96577495e+00
 3.96688820e+00 3.96209851e+00 3.94881520e+00 3.93597191e+00
 3.90807479e+00 3.88753343e+00 3.87025622e+00 3.86628286e+00
 3.86450740e+00 3.86552561e+00 3.86586011e+00 3.86779146e+00
 3.86962871e+00 3.87275065e+00 3.87357221e+00 3.87422812e+00
 3.87435822e+00 3.87382547e+00 3.87298224e+00 3.87104144e+00
 3.87139185e+00 3.87141953e+00 3.87133258e+00 3.86993565e+00
 3.86627551e+00 3.86221730e+00 3.85935658e+00 3.85768503e+00
 3.85742821e+00 3.85655879e+00 3.85538963e+00 3.85505686e+00
 3.85282704e+00 3.85067636e+00 3.84840659e+00 3.84199455e+00
 3.83273341e+00 3.80750391e+00 3.79739351e+00 3.79049480e+00
 3.78235138e+00 3.78265448e+00 3.78248780e+00 3.79585354e+00
 3.79782216e+00 3.80168751e+00 3.80399888e+00 3.80438724e+00
 3.80446075e+00 3.80468492e+00 3.80509538e+00 3.80369832e+00
 3.80272547e+00 3.80191830e+00 3.80071593e+00 3.80067686e+00
 3.79952255e+00 3.79937815e+00 3.79859596e+00 3.79835985e+00
 3.79593730e+00 3.79187716e+00 3.77670988e+00 3.77494485e+00
 3.76778630e+00 3.75417151e+00 3.75483932e+00 3.75759361e+00
 3.76458831e+00 3.77191283e+00 3.77764202e+00 3.77804091e+00
 3.77875098e+00 3.77971074e+00 3.77997216e+00 3.78041643e+00
 3.78032914e+00 3.77840021e+00 3.77763478e+00 3.77879455e+00
 3.77986603e+00 3.77870000e+00 3.77786991e+00 3.77765349e+00
 3.77774268e+00 3.77665873e+00 3.77618941e+00 3.77138773e+00
 3.76963046e+00 3.76575680e+00 3.76232838e+00 3.76497761e+00
 3.76776396e+00 3.77902872e+00 3.78332581e+00 3.78545462e+00
 3.78605138e+00 3.78645851e+00 3.78644215e+00 3.78801541e+00
 3.78842667e+00 3.78877170e+00 3.78892287e+00 3.78785162e+00
 3.78685515e+00 3.78714651e+00 3.78737574e+00 3.78541638e+00
 3.78386973e+00 3.78426630e+00 3.78751723e+00 3.79477825e+00
 3.80132665e+00 3.80683042e+00 3.80754462e+00 3.80749406e+00
 3.80788783e+00 3.80830202e+00 3.80784211e+00 3.80768669e+00
 3.80797143e+00 3.80898742e+00 3.80999671e+00 3.81017097e+00
 3.80929057e+00 3.80992168e+00 3.81304095e+00 3.81355803e+00
 3.81510001e+00 3.81495842e+00 3.81594285e+00 3.81691555e+00
 3.81708011e+00 3.81668162e+00 3.81696663e+00 3.82091093e+00
 3.82362382e+00 3.82530762e+00 3.82689099e+00 3.82897595e+00
 3.82945205e+00 3.83005442e+00 3.83020996e+00 3.83181849e+00
 3.83232033e+00 3.83293110e+00 3.84001148e+00"""
#coeff = [float(element) for element in copied.split()]
coeff = [50] * len(x)

#After


In [ ]:
f = lambda x, y: ((x-y)**2)/x
tempCoeff = coeff
def resid2(c):
  global tempCoeff
  tempCoeff = c
  predictedY = [sum(element) for element in np.multiply(np.array(x).T, np.array(c))]
  #print(predictedY)
  total = sum(f(y, predictedY))
  return total


In [ ]:
print(coeff)

In [ ]:
#Actually optimizing things
for i in range(10):
  coeff = tempCoeff
  result = spo.least_squares(resid2, coeff, verbose = 2, max_nfev = 1000, bounds = (0, 100000000))
coeffArchive = tempCoeff
print(result.x)
print(result.fun)

#Visualization


In [ ]:
plt.hist(np.arange(54,77,.25), bins = np.arange(54,77,.25), weights = y);
#print(np.shape(sum(np.multiply(np.array(x).T, coeff))));
plt.hist(np.arange(54, 77, .25), bins = np.arange(54, 77, .25), weights = sum(np.multiply(np.array(x).T, coeff).T));

In [ ]:
#makes a matplotlib rectangle object based on the coordinates of each corner.
def makeRect(s, c):
  return Rectangle((s[0], s[2]), s[1]-s[0], s[3]-s[2], color = c)

#takes a dictionary and plots all of the various rectangles
def visualizeRectangles(d, cs, r2):
  norm = colors.Normalize(-6e+03, 6e+03)


  fig, ax = plt.subplots()
  for key in d.keys():
    ax.add_patch(makeRect(d[key], cs[key]))

  bound = 180
  ax.set(xlim = (-1 * bound, bound), ylim = (-1*bound, bound))
  plt.xlabel("φ/phi (deg)")
  plt.ylabel("Ψ/psi (deg)")
  plt.title(f"Threonine Data: R²={round(r2, 3)}")
  fig.set_size_inches(6, 6)
  fig.colorbar(cm.ScalarMappable(norm=norm, cmap=cm.RdBu))
  plt.show()

In [ ]:
print(coeff)

In [ ]:
print(np.all(rawInput[648] == x[235]))
#print(x[235])

#print(rawDict["R36"])
#print(x[235])
for key in rawDict.keys():
  if np.all(rawDict[key] == x[235]):
    print(key)

In [ ]:
norm = colors.Normalize(-6e+03, 6e+03)
rawDict = {}
visDict = {}
cDict = {}

for i in range(int(180/xScale)):
    for j in range(int(360/yScale)):
      name = f"{chr(i+65)}{(j+1):02d}"
      rawDict[name] = rawInput[i*int(360/yScale) + j]
      visDict[name] = (i*xScale-180, (i+1)*xScale-180, 180-(j+1)*yScale, 180-j*yScale)
rawDict["rlhelix"] = rawInput[int(360*180/xScale/yScale)]
visDict["rlhelix"] = (0, 180, -180, 180)
for i in range(len(x)):
  for key in rawDict.keys():
    if np.all(rawDict[key] == x[i]):
      cDict[key] = cm.RdBu(norm(coeff[i]))

#print(cDict)
visualizeRectangles(visDict, cDict, 0)

In [ ]:
norm = colors.Normalize(0, 4)
rawDict = {}
visDict = {}
cDict = {}

for i in range(int(180/xScale)):
    for j in range(int(360/yScale)):
      name = f"{chr(i+65)}{(j+1):02d}"
      rawDict[name] = rawInput[i*int(360/yScale) + j]
      visDict[name] = (i*xScale-180, (i+1)*xScale-180, 180-(j+1)*yScale, 180-j*yScale)
rawDict["rturn"] = rawInput[648]
visDict["rturn"] = (0, 180, -180, 180)
for i in range(len(x)):
  for key in rawDict.keys():
    if np.all(rawDict[key] == x[i]):
      cDict[key] = cm.RdBu(norm(np.log10(coeff[i])))

#print(cDict)
visualizeRectangles(visDict, cDict, 0)

In [ ]:
norm = colors.Normalize(-2.7e+07, 2.7e+07)
visDict = {}
colorArray = {}
valDict = {}
for i in range(int(180/xScale)):
  for j in range(int(360/yScale)):
    visDict[f"{chr(i+65)}{(j+1):02d}"] = (i*xScale-180, (i+1)*xScale-180, 180-(j+1)*yScale, 180-j*yScale)
    val = float(tempArr[int((i)*360/yScale + (j))][1])
    #print(f"{np.log(abs(val))} / {val}")
    valDict[f"{chr(i+65)}{(j+1):02d}"] = val
    colorArray[f"{chr(i+65)}{(j+1):02d}"] = cm.RdBu(norm(val))

visDict["rturni"] = (0, 180, -180, 180)
#print(visDict)
colorArray["rturni"] = cm.RdBu(float(tempArr[-1][1]))
#print(valDict)
visualizeRectangles(visDict, colorArray, float(r))

print(colorArray)


#otherVal = min(max(np.log(val*-1)*-1/4,0),1)
#otherVal = min(max(np.log(float(tempArr[-1][1]))*-1/4,0),1)

#for explaining process

In [ ]:
q = np.array(([1, 2], [3, 4], [5, 6]))
#print(q)

r = np.array((1, 2, 3))
#print(r)
#print(q.T)

print((np.multiply(q.T, r)))
print(sum(np.multiply(q.T, r).T))

In [ ]:
np.set_printoptions(threshold=np.inf)
print(np.shape(np.array(x)))
print(sum(np.multiply(np.array(x).T, coeff)))
np.set_printoptions(threshold=1000)

In [ ]:
i = 1
print(np.shape(x))
plt.hist(np.arange(54,77,.25), bins = np.arange(54,77,.25), weights = x[42]);
#for element in x:
  #plt.hist(np.arange(54,77,.25), bins = np.arange(54,77,.25), weights = element, alpha = i);
